In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers
import os
import numpy as np
from PIL import Image

In [2]:
# Data Loading and Preprocessing
def load_data(real_images_folder, cartoon_images_folder, image_size, batch_size):
    real_images = []
    cartoon_images = []
    
    real_files = os.listdir(real_images_folder)
    cartoon_files = os.listdir(cartoon_images_folder)
    
    for i in range(len(real_files)):
        real_file = os.path.join(real_images_folder, real_files[i])
        cartoon_file = os.path.join(cartoon_images_folder, cartoon_files[i])
        
        real_image = Image.open(real_file).resize((image_size, image_size))
        cartoon_image = Image.open(cartoon_file).resize((image_size, image_size))
        
        real_images.append(np.array(real_image) / 255.0)
        cartoon_images.append(np.array(cartoon_image) / 255.0)
        
    real_images = np.array(real_images)
    cartoon_images = np.array(cartoon_images)
    
    return real_images, cartoon_images

In [3]:
# Generator Model
def build_generator(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, kernel_size=3, strides=1, padding='same')(inputs)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2DTranspose(32, kernel_size=3, strides=1, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    outputs = layers.Conv2D(3, kernel_size=3, strides=1, padding='same', activation='sigmoid')(x)  # Output 3 channels for RGB image
    return models.Model(inputs, outputs)

In [4]:
# Discriminator Model
def build_discriminator(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, kernel_size=3, strides=2, padding='same')(inputs)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return models.Model(inputs, outputs)

In [5]:
# Loss Functions
bce_loss = losses.BinaryCrossentropy(from_logits=True)

# Define input shape
image_size = 128
input_shape = (image_size, image_size, 3)

# Initialize Models
generator = build_generator(input_shape)
discriminator = build_discriminator(input_shape)

# Compile Discriminator
discriminator.compile(loss=bce_loss,
                      optimizer=optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Freeze Discriminator Weights
discriminator.trainable = False

In [6]:
# Define GAN Model
gan_input = layers.Input(shape=input_shape)
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)
gan = models.Model(gan_input, gan_output)

# Compile GAN
gan.compile(loss=bce_loss,
            optimizer=optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

In [1]:
# Training Loop
batch_size = 32
num_epochs = 50
real_images_folder =r'C:\Users\aruns\Downloads\realimages'
cartoon_images_folder = r'C:\Users\aruns\Downloads\cartoon'

real_images, cartoon_images = load_data(real_images_folder, cartoon_images_folder, image_size, batch_size)

num_batches = min(len(real_images), len(cartoon_images)) // batch_size

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}", flush=True)
    
    for batch_index in range(num_batches):
        start_index = batch_index * batch_size
        end_index = start_index + batch_size
        real_batch = real_images[start_index:end_index]
        cartoon_batch = cartoon_images[start_index:end_index]

        # Train Discriminator
        fake_cartoon_images = generator.predict(real_batch)
        discriminator_loss_real = discriminator.train_on_batch(cartoon_batch, tf.ones((len(real_batch), 1)))
        discriminator_loss_fake = discriminator.train_on_batch(fake_cartoon_images, tf.zeros((len(real_batch), 1)))
        discriminator_loss = 0.5 * (discriminator_loss_real + discriminator_loss_fake)

        # Train Generator
        generator_loss = gan.train_on_batch(real_batch, tf.ones((len(real_batch), 1)))

        print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_index+1}/{num_batches}, "
              f"Discriminator Loss: {discriminator_loss}, Generator Loss: {generator_loss}")

NameError: name 'load_data' is not defined

In [9]:
# Save the trained model in native Keras format
generator.save('gan_generator_model1.keras')

In [1]:
import tkinter as tk
from tkinter import filedialog
import os
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image, ImageTk

In [2]:
# Load the saved generator model
generator = tf.keras.models.load_model('gan_generator_model1.keras')

In [3]:
# Function to preprocess and resize the real image
def preprocess_and_resize_image(image_path, target_size=(128, 128)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Resize the image
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return np.expand_dims(image, axis=0)  # Add batch dimension

In [4]:
# Function to convert real image to cartoon image using the generator model
def convert_to_cartoon(real_image):
    cartoon_image = generator.predict(real_image)
    return cartoon_image

In [5]:
def open_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        # Preprocess and resize the uploaded image
        unseen_real_image = preprocess_and_resize_image(file_path)
        # Convert the resized image to a cartoon image
        cartoon_image = convert_to_cartoon(unseen_real_image)
        # Display the original and cartoon images
        original_image = Image.open(file_path)
        original_image.thumbnail((400, 400))
        cartoon_image = Image.fromarray((cartoon_image[0] * 255).astype(np.uint8))
        cartoon_image.thumbnail((400, 400))
        original_image_tk = ImageTk.PhotoImage(original_image)
        cartoon_image_tk = ImageTk.PhotoImage(cartoon_image)
        original_label.config(image=original_image_tk)
        original_label.image = original_image_tk
        cartoon_label.config(image=cartoon_image_tk)
        cartoon_label.image = cartoon_image_tk
        # Set size of cartoon label to match original label
        cartoon_label.config(width=original_image_tk.width(), height=original_image_tk.height())


In [6]:
# Create a Tkinter window
root = tk.Tk()
root.title("Image Cartooning System")

''

In [7]:
# Create a frame
frame = tk.Frame(root)
frame.pack(padx=10, pady=10)

In [8]:
# Create labels for original and cartoon images
original_label = tk.Label(frame)
original_label.pack(side="left", padx=10, pady=10)
cartoon_label = tk.Label(frame)
cartoon_label.pack(side="right", padx=10, pady=10)

In [ ]:
# Create a button to open an image
open_button = tk.Button(root, text="Open Image", command=open_image)
open_button.pack(pady=10)

root.mainloop()